In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pprint import pprint
import requests
import time
from scipy.stats import linregress
import os
import csv

#Declare columns/variables to keep and create a summary dataframe
#links to data dictionaries for dui studies (help decided relevant columns)
#https://www.nhtsa.gov/sites/nhtsa.dot.gov/files/documents/2007_nrsdatadictionary.pdf
#https://www.nhtsa.gov/sites/nhtsa.dot.gov/files/documents/nrs_2013datadictionary.pdf

#census data link to retrieve population information per state
#https://www.census.gov/quickfacts/fact/table/US/PST040219

# Output File (CSV), if needed
output_2007data_file = "output_data_file/dui2007stats.csv"
output_2013data_file = "output_data_file/dui2013stats.csv"

# File to Load (**Remember to Change These file names to DUI saved file names and the path where they are finalized)
dui_2013_data = "nrs_2013_combined.csv"
dui_2007_data = "nrs_2007_final.csv"

col2007_list = ["STATE", "DATE","NRS_ZIPCODE","session","NRS_RACE","NRS_DSEX","NRS_AGE","NRS_SCHOOL","NRS_EMPLOY","NRS_VTYPE",
            "NRS_FROM","NRS_HEADED", "NRS_BETMI", "NRS_TODRK","resulbac2"]

col2013_list = ["State","MyZipCodeIs","Time_Stamp","data_session_original","race_n", "DriverAgeYears","EducationLevel","AreYouEmployed",
                "VehicleType", "FromWhere", "ToWhere","MilesTraveling", "AlcoholToday", "bac_from_blood"]

# Read DUI 2007 and 2013 Data Files and store into Pandas DataFrames(can't start with a number)
dui2013_data = pd.read_csv(dui_2013_data,low_memory=False, usecols=col2013_list)
dui2007_data = pd.read_csv(dui_2007_data,low_memory=False, usecols=col2007_list)

In [2]:
#Break out 2007 NRS_EMPLOY column to match 2013 data that is 2 columns of same info

Student_2007 = []

dui2007_data["Student_2007"] = dui2007_data.apply(lambda row: "Student" if row.NRS_EMPLOY == "Student" else "No", axis = 1)

In [3]:
#Rename column headers for uniformity

dui2007_data.rename(columns = {"STATE":"State_2007", "NRS_ZIPCODE":"Zipcode_2007", "DATE":"Date_2007","session":"Day or Night?_2007",
                               "NRS_RACE":"Race_2007","NRS_DSEX":"Gender_2007","NRS_AGE":"Age Range_2007","NRS_SCHOOL":"Education Level_2007",
                               "NRS_EMPLOY":"Employed_2007","NRS_VTYPE":"Vehicle Type_2007", "NRS_FROM":"From Where_2007", "NRS_HEADED":"To Where_2007",
                               "NRS_BETMI":"Trip distance_2007", "NRS_TODRK":"Alcohol Today_2007", "resulbac2":"Blood Alcohol Content_2007"}, inplace = True)

In [4]:
dui2007_data

,Day or Night?_2007,State_2007,Date_2007,From Where_2007,To Where_2007,Trip distance_2007,Alcohol Today_2007,Age Range_2007,Zipcode_2007,Education Level_2007,Employed_2007,Race_2007,Vehicle Type_2007,Gender_2007,Blood Alcohol Content_2007,Student_2007
0,3,TN,11/3/2007,NaN,NaN,NaN,NaN,#NULL!,NaN,NaN,NaN,NaN,NaN,NaN,0,No
1,1,IL,NaN,NaN,NaN,NaN,NaN,#NULL!,NaN,NaN,NaN,NaN,NaN,NaN,#NULL!,No
2,1,IN,NaN,NaN,NaN,NaN,NaN,#NULL!,NaN,NaN,NaN,NaN,NaN,NaN,0,No
3,1,MI,10/12/2007,NaN,NaN,NaN,NaN,#NULL!,NaN,NaN,NaN,NaN,NaN,NaN,0,No
4,5,IA,NaN,NaN,NaN,NaN,NaN,#NULL!,NaN,NaN,NaN,NaN,NaN,NaN,#NULL!,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11115,2,IN,NaN,Sport or rec facility / park,Home (own home),0 - 5,No,48,46373,Some college,Employed/self-employed,White,Car,1.0,0,No
11116,3,PA,NaN,Restaurant / eating place,Home (own home),10-Jun,Yes,39,19038,College graduate,Employed/self-employed,White,Car,2.0,#NULL!,No
11117,5,FL,NaN,Other,Home (own home),20-Nov,No,19,33065,Some college,Student,White,Car,2.0,#NULL!,Student
11118,1,PA,NaN,Restaurant / eating place,Home (own home),16 - 20,NaN,#NULL!,19312,College graduate,Retired,White,Car,2.0,#NULL!,No


In [5]:
#Clean up null values

dui2007_no_answer = dui2007_data.replace(np.nan,"Not answered")
dui2007_clean = dui2007_no_answer.replace(["#NULL!"],"Not answered")
dui2007_clean

,Day or Night?_2007,State_2007,Date_2007,From Where_2007,To Where_2007,Trip distance_2007,Alcohol Today_2007,Age Range_2007,Zipcode_2007,Education Level_2007,Employed_2007,Race_2007,Vehicle Type_2007,Gender_2007,Blood Alcohol Content_2007,Student_2007
0,3,TN,11/3/2007,Not answered,Not answered,Not answered,Not answered,Not answered,Not answered,Not answered,Not answered,Not answered,Not answered,Not answered,0,No
1,1,IL,Not answered,Not answered,Not answered,Not answered,Not answered,Not answered,Not answered,Not answered,Not answered,Not answered,Not answered,Not answered,Not answered,No
2,1,IN,Not answered,Not answered,Not answered,Not answered,Not answered,Not answered,Not answered,Not answered,Not answered,Not answered,Not answered,Not answered,0,No
3,1,MI,10/12/2007,Not answered,Not answered,Not answered,Not answered,Not answered,Not answered,Not answered,Not answered,Not answered,Not answered,Not answered,0,No
4,5,IA,Not answered,Not answered,Not answered,Not answered,Not answered,Not answered,Not answered,Not answered,Not answered,Not answered,Not answered,Not answered,Not answered,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11115,2,IN,Not answered,Sport or rec facility / park,Home (own home),0 - 5,No,48,46373,Some college,Employed/self-employed,White,Car,1,0,No
11116,3,PA,Not answered,Restaurant / eating place,Home (own home),10-Jun,Yes,39,19038,College graduate,Employed/self-employed,White,Car,2,Not answered,No
11117,5,FL,Not answered,Other,Home (own home),20-Nov,No,19,33065,Some college,Student,White,Car,2,Not answered,Student
11118,1,PA,Not answered,Restaurant / eating place,Home (own home),16 - 20,Not answered,Not answered,19312,College graduate,Retired,White,Car,2,Not answered,No


In [6]:
#Rename column headers for uniformity

dui2013_data.rename(columns = {"State":"State_2013","MyZipCodeIs":"Zipcode_2013", "Time_Stamp":"Date_2013","data_session_original":"Day or Night?_2013",
                               "race_n":"Race_2013", "DriverAgeYears":"Age Range_2013","EducationLevel":"Education Level_2013","AreYouEmployed":"Employed_2013",
                               "VehicleType":"Vehicle Type_2013", "FromWhere":"From Where_2013", "ToWhere":"To Where_2013","MilesTraveling":"Trip distance_2013",
                               "AlcoholToday":"Alcohol Today_2013", "bac_from_blood":"Blood Alcohol Content_2013"}, inplace = True)

In [7]:
#Clean up null values

dui2013_no_answer = dui2013_data.replace(np.nan,"Not answered")
dui2013_clean = dui2013_no_answer.replace(["#NULL!"],"Not answered")
dui2013_clean

,Day or Night?_2013,State_2013,Date_2013,Race_2013,Vehicle Type_2013,From Where_2013,To Where_2013,Trip distance_2013,Alcohol Today_2013,Age Range_2013,Zipcode_2013,Education Level_2013,Employed_2013,Blood Alcohol Content_2013
0,3,CA,12/7/2013 1:36,8,Car,Other,Other,0-5,No,43,93003,High school graduate,Employed Full-time,Not answered
1,3,CA,12/7/2013 1:56,1,Car,Other,Own home,More than 20,No,21,93033,Some college - no degree,Employed Full-time,Not answered
2,3,CA,12/7/2013 2:17,8,Car,Other,Own home,0-5,Yes,23,93035,Some college - no degree,Other __________________________,Not answered
3,3,CA,12/7/2013 2:34,8,Car,Someone else's home,Own home,0-5,No,19,93030,Some college - no degree,Employed Part-time,0
4,3,CA,12/7/2013 2:41,Not answered,SUV/ Crossover,Not answered,Not answered,Not answered,Not answered,Not answered,Not answered,Not answered,Not answered,Not answered
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11317,4,UT,2/8/2014 23:46,1,Car,Work,Own home,20-Nov,Not answered,56,84010,Associate's degree,Employed Full-time,0
11318,5,UT,2/9/2014 1:29,1,SUV/ Crossover,Someone else's home,Own home,20-Nov,Not answered,18,84054,High school graduate,Employed Part-time,0
11319,5,UT,2/9/2014 1:43,1,Car,Someone else's home,Own home,20-Nov,Not answered,22,84037,Some college - no degree,Employed Part-time,0
11320,5,UT,2/9/2014 2:03,1,Car,Store or gas station,Other,10-Jun,No,20,84606,Some college - no degree,Employed Part-time,0


In [8]:
dui2007_data.to_csv(output_2007data_file, index=False)
dui2013_data.to_csv(output_2013data_file, index=False)

In [9]:
# Combine the data into a single dataset

dui_data_complete = pd.merge(dui2013_clean,dui2007_clean, how="left", on=["State_2013","State_2007"])

KeyError: 'State_2013'

In [ ]:
dui_data_complete.head()

In [ ]:
# Create an overview table grouped by State

per2007_state = dui2007_clean.set_index("State").groupby(["State"])
per2007_state.head()

In [ ]:
# Create an overview table grouped by State

per2013_state = dui2013_clean.set_index("State").groupby(["State"])
per2013_state.head()